# LU decomposition - introduction

The Gaussian elimination enshrines the LU decompositon. The LU decomposition is a factorization process that describes a matrix $\mathbf{A}$ as a product of a unit triangular matrix $\mathbf{L}$ (i.e., all the entries of its main diagonal are equal to one) and an upper triangular matrix $\mathbf{U}$.

For details, see:

* Turing, A.M. (1948). "Rounding-Off Errors in Matrix Processes". The Quarterly Journal of Mechanics and Applied Mathematics. 1: 287–308. doi: [10.1093/qjmam/1.1.287](https://doi.org/10.1093/qjmam/1.1.287)

* A. Schwarzenberg-Czerny (1995). "On matrix factorization and efficient least squares solution". Astronomy and Astrophysics Supplement. 110: 405-410. Bibliographic Code ADS: [1995A&AS..110..405S](http://adsabs.harvard.edu/abs/1995A%26AS..110..405S)

* Golub, G. H. and C. F. Van Loan, (2013), Matrix computations, 4th edition, Johns Hopkins University Press, ISBN 978-1-4214-0794-4.

Let's first recall the $3 \times 3$ linear system $\mathbf{A}\mathbf{x} = \mathbf{y}$ presented in our last classes:

In [ ]:
import numpy as np
from scipy.linalg import lu

In [ ]:
A = np.array([[2.,1.,-1.],
              [-3.,-1.,2.],
              [-2.,1.,2.]])

In [ ]:
y = np.array([8., -11., -3.])

The solution of this system is given by:

In [ ]:
x = np.linalg.solve(A,y)

In [ ]:
print(x)

Let's remember the solution of this linear system by using the Gaussian elimination without pivoting.

<a id='eq1'></a>
$$
\begin{align}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \tag{1a} \\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{y}^{(0)} \tag{1b} \\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{y}^{(1)} \tag{1c}
\end{align}$$

where $\mathbf{M}^{(k)}$ is a matrix given by

<a id='eq2'></a>
$$
\mathbf{M}^{(k)} = \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \: , \tag{2}
$$

$\mathbf{u}^{(k-1)}$ is a $3 \times 1$ vector with all elements equal to $0$, except the $(k-1)$th element, which is equal to $1$, and $\mathbf{t}^{(k)}$ is a $3 \times 1$ vector whose $i$th element $t_{i}^{(k)}$ is given by:

<a id='eq3'></a>
$$
t_{i}^{(k)} = \begin{cases} 0 & \quad \text{if } i < k \\\\ \dfrac{a^{(k-1)}_{i(k-1)}}{a^{(k-1)}_{(k-1)(k-1)}} & \quad \text{if } i \ge k\\ \end{cases} \: , \tag{3}
$$

where $a^{(k-1)}_{ij}$ is the $ij$ element of the matrix $\mathbf{A}^{(k-1)}$.

At the end of this algorithm, the original matrix $\mathbf{A}$ ([equation 1a](#eq1)) is transformed into an upper triangular matrix $\mathbf{A}^{(2)}$ ([equation 1c](#eq1)).

Now, let's analyze the matrices $\mathbf{M}^{(1)}$ and $\mathbf{M}^{(2)}$. They are given by:

<a id='eq4a'></a>
$$
\begin{split}
\mathbf{M}^{(1)} &= \mathbf{t}^{(1)} \cdot \left( \mathbf{u}^{(0)} \right)^{\top}  \\
&= \left[ 
\begin{array}{c}
0 \\
t_{2}^{(1)} \\
t_{3}^{(1)}
\end{array} \right] \cdot 
\left[ \begin{array}{ccc}
1 & 0 & 0
\end{array} \right] \\
&= \left[ \begin{array}{ccc}
0 & 0 & 0 \\
t_{2}^{(1)} & 0 & 0 \\
t_{3}^{(1)} & 0 & 0
\end{array} \right]
\end{split} \tag{4a}
$$

and, similarly, 

<a id='eq4b'></a>
$$
\begin{split}
\mathbf{M}^{(2)} &= \mathbf{t}^{(2)} \cdot \left( \mathbf{u}^{(1)} \right)^{\top}  \\
&= \left[ 
\begin{array}{c}
0 \\
0 \\
t_{3}^{(2)}
\end{array} \right] \cdot 
\left[ \begin{array}{ccc}
0 & 1 & 0
\end{array} \right] \\
&= \left[ \begin{array}{ccc}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & t_{3}^{(2)} & 0
\end{array} \right]
\end{split} \quad . \tag{4b}
$$

Notice that, according to the algorithm, the matrix $\mathbf{A}^{(2)}$ ([equation 1c](#eq1)) can be written as follows:

<a id='eq5a'></a>
$$
\mathbf{A}^{(2)} = \mathbf{Q} \, \mathbf{A} \quad , \tag{5a}
$$

and

<a id='eq5b'></a>
$$
\mathbf{Q} = \mathbf{Q}^{(2)} \, \mathbf{Q}^{(1)} \quad , \tag{5b}
$$

were the Gauss transformations $\mathbf{Q}^{(k)}$ ([equation 5b](#eq5b)), $k = 1, \dots, N-1$, are given by:

<a id='eq6'></a>
$$
\begin{split}
\mathbf{Q}^{(1)} &= \left( \mathbf{I} - \mathbf{M}^{(1)} \right) \\
\mathbf{Q}^{(2)} &= \left( \mathbf{I} - \mathbf{M}^{(2)} \right)
\end{split} \quad . \tag{6}
$$

Then, the original matrix $\mathbf{A}$ ([equation 1a](#eq1)) can be written as follows:

<a id='eq7a'></a>
$$
\mathbf{A} = \mathbf{Q}^{-1} \, \mathbf{A}^{(2)} \quad , \tag{7a}
$$

and

<a id='eq7b'></a>
$$
\mathbf{Q}^{-1} = \mathbf{Q}^{(-1)} \, \mathbf{Q}^{(-2)} \quad , \tag{7b}
$$

where $\mathbf{Q}^{(-k)}$ represent the inverse of a Gauss transformation $\mathbf{Q}^{(k)}$ ([equation 6](#eq6)), $k = 1, \dots, N-1$. We know that $\mathbf{A}^{(2)}$ ([equation 1c](#eq1)) is an upper triangular matrix. However, what about the matrix $\mathbf{Q}^{-1}$ ([equation 7b](#eq7b))? To answer this question, let's define a generic $\mathbf{Q}^{(-k)}$:

<a id='eq8'></a>
$$
\begin{split}
\mathbf{Q}^{(-k)} &=
\mathbf{I} + \mathbf{M}^{(k)} \\
&= \mathbf{I} + \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top}
\end{split} \: . \tag{8}
$$

We can verify that $\mathbf{Q}^{(-k)}$ ([equation 8](#eq8)) is actually the inverse of a Gauss transformation $\mathbf{Q}^{(k)}$ ([equation 6](#eq6)) by computing the product $\mathbf{Q}^{(k)} \mathbf{Q}^{(-k)}$:

<a id='eq9'></a>
$$
\begin{split}
\mathbf{Q}^{(k)} \mathbf{Q}^{(-k)}
&= \left[ \mathbf{I} - \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \right] \,
\left[ \mathbf{I} + \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \right] \\
&= \mathbf{I} - \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top}
+ \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} -
\mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \, \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \\
&= \mathbf{I} - 
\mathbf{t}^{(k)} \cdot 
\underbrace{ \left( \mathbf{u}^{(k-1)} \right)^{\top} \mathbf{t}^{(k)} }_{= \, 0}
\cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \\
&= \mathbf{I}
\end{split} \quad . \tag{9}
$$

The third row of ([equation 9](#eq9)) uses the fact that $\left( \mathbf{u}^{(k-1)} \right)^{\top} \mathbf{t}^{(k)} = 0$. This property can be easily verified by noting that, while the $(k-1)$th element of $\mathbf{u}^{(k-1)}$ is its only nonzero element, the Gauss multipliers $t_{i}^{(k)}$, $k = 0, \dots, k-1$, forming the Gauss vector $\mathbf{t}^{(k)}$ are equal to zero.

By using this result, we can write the matrix $\mathbf{Q}^{-1}$ (equations [7a](#eq7a) and [7b](#eq7b)) as follows:

<a id='eq10'></a>
$$
\begin{split}
\mathbf{Q}^{-1}
&= \left[ \mathbf{I} + \mathbf{t}^{(1)} \cdot \left( \mathbf{u}^{(0)} \right)^{\top} \right]
   \left[ \mathbf{I} + \mathbf{t}^{(2)} \cdot \left( \mathbf{u}^{(1)} \right)^{\top} \right] \\
&= \mathbf{I} + \mathbf{t}^{(1)} \cdot \left( \mathbf{u}^{(0)} \right)^{\top} +
   \mathbf{t}^{(2)} \cdot \left( \mathbf{u}^{(1)} \right)^{\top} + 
   \mathbf{t}^{(1)} \cdot 
   \underbrace{ \left( \mathbf{u}^{(0)} \right)^{\top} \mathbf{t}^{(2)}}_{= \, 0}
   \cdot \left( \mathbf{u}^{(1)} \right)^{\top} \\
&= \mathbf{I} + \mathbf{t}^{(1)} \cdot \left( \mathbf{u}^{(0)} \right)^{\top} +
   \mathbf{t}^{(2)} \cdot \left( \mathbf{u}^{(1)} \right)^{\top} \\
&= 
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
t_{2}^{(1)} & 0 & 0 \\
t_{3}^{(1)} & 0 & 0
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & t_{3}^{(2)} & 0
\end{array} \right] \\
&=
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
t_{2}^{(1)} & 1 & 0 \\
t_{3}^{(1)} & t_{3}^{(2)} & 1
\end{array} \right]
\end{split} \quad\quad . \tag{10}
$$

[Equation 10](#eq10) shows that $\mathbf{Q}^{-1}$ (equations [7a](#eq7a) and [7b](#eq7b)) is a unit **L**ower triangular matrix containing the Gauss multipliers. Because of that, $\mathbf{Q}^{-1}$ is commonly defined as $\mathbf{L}$. Similarly, the **U**pper triangular matrix $\mathbf{A}^{(2)}$ is defined as $\mathbf{U}$ and the original matrix $\mathbf{A}$ is factored as follows:

<a id='eq11'></a>
$$
\mathbf{A} = \mathbf{L} \, \mathbf{U} \: . \tag{11}
$$

The mathematical development presented here is valid for $3 \times 3$ matrices, however, it can be easily generalized for $N \times N$ matrices.

## Solving a linear system by using the LU decomposition

Once the triangular matrices $\mathbf{L}$ and $\mathbf{U}$ are calculated ([equation 11](#eq11)), we may use them to solve a linear system $\mathbf{A} \mathbf{x} = \mathbf{y}$. Let's first substitute the LU decomposition into the linear system:

<a id='eq12'></a>
$$
\begin{split}
\mathbf{A} \mathbf{x} &= \mathbf{y} \\
\mathbf{L} \mathbf{U} \mathbf{x} &= \mathbf{y}
\end{split} \quad . \tag{12}
$$

This equation shows that the original linear system can be represented by two triangular systems:

<a id='eq13'></a>
$$
\begin{align}
\mathbf{L}\mathbf{w} &= \mathbf{y} \tag{13a} \\
\mathbf{U}\mathbf{x} &= \mathbf{w} \tag{13b}
\end{align}
$$

where $\mathbf{w}$ is a dummy vector. Therefore, the linear system ([equation 12](#eq12)) can be solved in two steps: 

1. Solve the lower triangular system for $\mathbf{w}$ ([equation 13a](#eq13)) and
2. Use it to solve the upper triangular system for $\mathbf{x}$ ([equation 13b](#eq13)) .

## Existence of the LU decomposition

The LU decomposition may not exist. Based on our previous classes, it is relatively easy to notice that the existence of the LU decomposition requires all the pivots be nonzero. However, there is a formal definition of the conditions under which the LU decomposition exists (Golub and Van Loan, 2013):

**THEOREM:** If $\mathbf{A} \in \mathbb{R}^{N \times N}$ and $\text{det} \left( \mathbf{A} \left[1:k \, , 1:k \right] \right) \ne 0$ for $k = 1 : N-1$, then there exists a lower triangular matrix $\mathbf{L} \in \mathbb{R}^{N \times N}$ and an upper triangular matrix $\mathbf{U} \in \mathbb{R}^{N \times N}$ such that $\mathbf{A} = \mathbf{L} \mathbf{U}$. If this is the case and $\mathbf{A}$ is nonsingular, then the factorization is unique and $\text{det} \left( \mathbf{A} \right) = u_{11} \cdots u_{NN}$.

The LU decomposition described above can be implemented similarly to the function `Gauss_elim` in the notebook `gauss-elim-pivoting` as follows:

    def lu_decomp(A, check_input=True):
        N = A.shape[0]
        if check_input is True:
            assert A.ndim == 2, 'A must be a matrix'
            assert A.shape[1] == N, 'A must be square'

        # create matrix C as a copy of A
        C = 

        for k = 1:N-1

            # assert the pivot is nonzero
            assert C[k-1,k-1] != 0., 'null pivot!'

            # calculate the Gauss multipliers and store them 
            # in the lower part of C
            C[k:,k-1] = 

            # zeroing of the elements in the (k-1)th column
            C[k:,k:] = 

        # return matrix C
        return C

This algorithm receives a square matrix $\mathbf{A}$ and returns a matrix $\mathbf{C}$ containing the triangular matrices $\mathbf{L}$ and $\mathbf{U}$. The elements of $\mathbf{L}$, except the unitary elements of its main diagonal, are stored below the main diagonal of $\mathbf{C}$. The elements of $\mathbf{U}$ are stored in the upper part of $\mathbf{C}$, including its main diagonal.

#### Testing the function `lu_decomp`

### Exercise

1. In your `my_functions.py` file, create a function called `lu_decomp` for implementing the LU decomposition presented above. The code must receive a matrix `A` and return the matrix `C` containing the matrices `L` (without the main diagonal) and `U`.

2. In your `my_functions.py` file, create a function called `lu_solve`. This function must receive the matrix `C` and a vector `y` and return the solution vector `x`. Function `lu_solve` must use the functions you have created for solving triangular systems.

3. In your `test_my_functions.py` file, create a test that defines a square matrix `A`, compute a matrix `C` by using the function `lu_decomp`, use `C` to create the triangular matrices `L` and `U` and verify the if condition `A = LU` is satisfied.

4. In your `test_my_functions.py` file, create two aditional tests. One test must compare the solution of the linear system obtained by `lu_solve` and the solution obtained by [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html). The second test must create a matrix `A0` and a vector `x0` and use them to compute a vector `A0x0 = y0`. Then, use the function `lu_solve` to compute a vector `x1` by solving the linear system. Finally, compare the computed vector `x1` and the expected vector `x0`.